In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import requests
import os
import alpaca_trade_api as tradeapi
import requests
import quandl
import yfinance as yf


%matplotlib inline

In [2]:
load_dotenv()

True

# First API used will be Alpaca, purely to get the list of tickers

Getting key from the 'os' package through the env files:

In [3]:
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

#verify
type(alpaca_api_key)
type(alpaca_secret_key)

str

In [4]:
#Create the Alpaca API object
alpaca = tradeapi.REST?

#define it with the parameters
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2",
    base_url='https://paper-api.alpaca.markets'
)

Init signature:
tradeapi.REST(
    key_id: str = None,
    secret_key: str = None,
    base_url: alpaca_trade_api.common.URL = None,
    api_version: str = None,
    oauth=None,
    raw_data: bool = False,
)
Docstring:      <no docstring>
Init docstring:
:param raw_data: should we return api response raw or wrap it with
                 Entity objects.
File:           /opt/anaconda3/envs/dev/lib/python3.7/site-packages/alpaca_trade_api/rest.py
Type:           type
Subclasses:     


In [5]:
#Format Current date as ISO format
begin = pd.Timestamp('2020-06-01', tz = 'America/New_York').isoformat()
ending = pd.Timestamp('2023-01-01', tz = 'America/New_York').isoformat()

## The list_assets() method in the alpaca API allows you to list every single asset on the databse

In [6]:
active_assets = alpaca.list_assets(status='active')  # you could leave out the status to also get the inactive ones

### We are choosing the NYSE as the universe of stocks, elimate all that aren't on the NYSE:

In [7]:
# sp = [x for x in active_assets if x.exchange == 'NYSE']

z = []

for x in active_assets:
    if x.exchange =='NYSE':
        z.append(x)

Append all the tickers into a list, tickers:

In [8]:
tickers = []

for i in z:
    tickers.append(i.symbol)


The length of our first investment universe:

In [9]:
len(tickers)

2824

# 2nd Packaged that will be used: yfinance.<br>

### The reason is that yfinance provides data that stretches much further back whilst Alpaca only goes to 2016. 

Split the ticker list into 2 due to computational difficulty:

In [10]:
def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

ticker_1, ticker_2 = split_list(tickers)

Run the first lot:

In [11]:
data = yf.download(ticker_1, interval='1mo', start='2002-01-01', rounding=True)

[*********************100%***********************]  1412 of 1412 completed

111 Failed downloads:
- IPVF.U: No timezone found, symbol may be delisted
- NFNT.U: No timezone found, symbol may be delisted
- RJAC.U: No timezone found, symbol may be delisted
- CHAA.U: No timezone found, symbol may be delisted
- CLAA.U: No timezone found, symbol may be delisted
- PORT.U: No timezone found, symbol may be delisted
- GRP.U: No timezone found, symbol may be delisted
- RITM.PRB: No timezone found, symbol may be delisted
- ATEK.U: No timezone found, symbol may be delisted
- CIG.C: No timezone found, symbol may be delisted
- BMAC.U: No timezone found, symbol may be delisted
- CELG.RT: No timezone found, symbol may be delisted
- CNDA.U: No timezone found, symbol may be delisted
- BH.A: No timezone found, symbol may be delisted
- EGGF.U: No timezone found, symbol may be delisted
- GIA.U: No timezone found, symbol may be delisted
- WRAC.U: No timezone found, symbol may be delisted
- JATT.U: No timezon

Run the second lot:

In [12]:
data1 = yf.download(ticker_2, interval='1mo', start='2002-01-01', rounding=True)

[*********************100%***********************]  1412 of 1412 completed

105 Failed downloads:
- MBAC.U: No timezone found, symbol may be delisted
- TRAQ.U: No timezone found, symbol may be delisted
- BF.A: No timezone found, symbol may be delisted
- LEN.B: No data found for this date range, symbol may be delisted
- MNTN.U: No timezone found, symbol may be delisted
- NFYS.U: No timezone found, symbol may be delisted
- WSO.B: No timezone found, symbol may be delisted
- GNL.PRB: No timezone found, symbol may be delisted
- AGAC.U: No timezone found, symbol may be delisted
- HHLA.U: No timezone found, symbol may be delisted
- GLP.PRB: No timezone found, symbol may be delisted
- MKC.V: No timezone found, symbol may be delisted
- DSX.PRB: No timezone found, symbol may be delisted
- BSAQ.U: No timezone found, symbol may be delisted
- LOKM.U: No timezone found, symbol may be delisted
- PDOT.U: No timezone found, symbol may be delisted
- FLME.U: No timezone found, symbol may be delisted
- AA

## **1st Universe:**<br>Below the dataframe 'prices_df' is all of the prices for 2824 stocks on the NYSE from 2002 and 2023

In [21]:
master_df1 = data['Adj Close']
master_df2 = data1['Adj Close']

prices_df = pd.concat([master_df1, master_df2], axis=1)


prices_df.tail(9)

,AAT,AC,ACDI,ACM,ADCT,AFL,AFT,AFTR,AFTR.U,AGCO,...,YOU,YPF,ZETA,ZIP,ZTO,ZTR,ZTS,ZUO,ZVIA,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-01 00:00:00-04:00,29.70,35.83,10.01,64.79,7.95,55.33,12.05,9.71,NaN,98.30,...,20.00,3.26,4.52,14.82,27.45,8.11,171.89,8.95,2.81,27.24
2022-07-01 00:00:00-04:00,30.23,40.90,10.07,71.53,7.41,57.30,12.57,9.73,NaN,108.48,...,25.32,3.71,5.35,17.53,25.59,8.71,182.55,8.51,3.40,28.95
2022-08-01 00:00:00-04:00,27.76,39.59,10.08,72.84,6.82,59.42,12.65,9.76,NaN,108.27,...,22.94,5.49,7.00,16.68,26.05,7.62,156.53,7.68,4.43,27.58
2022-09-01 00:00:00-04:00,25.72,36.76,10.14,68.08,4.82,56.20,11.99,9.77,NaN,95.99,...,22.86,6.21,6.61,16.50,24.03,6.23,148.29,7.38,4.32,24.50
2022-10-01 00:00:00-04:00,27.48,40.30,10.20,74.96,4.44,65.11,12.03,9.89,NaN,123.94,...,27.02,7.64,8.34,16.77,16.89,6.44,150.78,7.69,4.69,23.49
2022-11-01 00:00:00-04:00,29.29,40.45,10.25,84.82,3.66,71.93,12.50,9.96,NaN,132.47,...,31.08,8.39,8.38,16.56,25.07,6.95,154.14,7.68,4.27,24.21
2022-12-01 00:00:00-05:00,26.50,41.99,10.32,84.75,3.84,71.94,12.13,10.02,NaN,138.69,...,27.43,9.19,8.17,16.42,26.87,6.55,146.55,6.36,4.09,21.15
2023-01-01 00:00:00-05:00,28.46,38.05,10.36,87.09,5.11,73.50,13.02,10.10,NaN,138.13,...,31.39,11.95,9.08,19.64,28.51,6.90,165.49,7.92,4.48,21.86
2023-02-01 00:00:00-05:00,28.30,37.08,10.38,88.24,4.56,69.58,13.18,10.12,NaN,134.62,...,33.46,11.48,9.33,21.94,28.13,6.91,164.93,8.46,4.88,22.30


### Below is the 12 month momentum of these stocks

In [22]:
#IF MONTHLY
six_month_return = prices_df.pct_change(periods=12)
six_month_return = six_month_return.iloc[6:,:]
six_month_return.tail(5)

,AAT,AC,ACDI,ACM,ADCT,AFL,AFT,AFTR,AFTR.U,AGCO,...,YOU,YPF,ZETA,ZIP,ZTO,ZTR,ZTS,ZUO,ZVIA,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2022-10-01 00:00:00-04:00,-0.267591,0.111418,NaN,0.107892,-0.846844,0.227796,-0.166320,0.019588,NaN,0.062130,...,-0.395390,0.819048,0.009685,-0.396328,-0.424139,-0.333333,-0.300649,-0.648216,-0.580876,-0.352536
2022-11-01 00:00:00-04:00,-0.141559,0.026650,NaN,0.243330,-0.835580,0.344737,-0.158249,0.022587,NaN,0.258862,...,-0.021411,1.343575,0.006002,-0.401085,-0.206897,-0.258271,-0.304673,-0.612317,-0.441830,-0.309272
2022-12-01 00:00:00-05:00,-0.288017,-0.023488,0.047716,0.107264,-0.809901,0.239704,-0.172014,0.027692,NaN,0.249910,...,-0.125598,1.405759,-0.029691,-0.341620,-0.047838,-0.309800,-0.398498,-0.659529,-0.419858,-0.418956
2023-01-01 00:00:00-05:00,-0.208785,-0.152561,0.046465,0.273059,-0.678414,0.177319,-0.120270,0.047718,NaN,0.232313,...,0.271365,1.766204,-0.019438,-0.094514,-0.050932,-0.272152,-0.170351,-0.523752,-0.440000,-0.284217
2023-02-01 00:00:00-05:00,-0.225930,-0.052389,0.044266,0.224875,-0.726946,0.146104,-0.080251,0.048705,NaN,0.171424,...,0.242481,1.733333,-0.282859,0.098098,-0.044497,-0.243154,-0.148309,-0.442688,-0.219200,-0.314268


In [47]:
# #get current closing prices for tickers
# dataframe = alpaca.get_bars(
#     tickers,
#     timeframe,
#     start = begin,
#     end = ending,
# ).df.drop(columns=['open','high','low','volume','trade_count','vwap'])


In [161]:
# dataframe.index = dataframe.index.date

In [163]:
# new_df = dataframe.pivot(columns = 'symbol', values = 'close')

1) Collect the Data
    - A Dataframe with all stocks: prices, market capitlization (monthly or 6 monthly)

2) Clean the Data
    - Remove Na's, remove stocks that don't have data back to 2000 (Decide what the threshold is)

3) Write the code which every 6 month long's the top 10% of stocks based on momentum



5) 
Benchmark
- Get the prices
- calculated the returns


Pyfolio:
- annualized return
- annualized volatility
- concat the portfolio returns and benchmark

